In [ ]:
# Import necessary libraries

import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score
import random
from torch.nn.utils.rnn import *


from torch.utils import data
from torchvision import transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import time

np.random.seed(11785)
torch.manual_seed(11785)

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
sys.version
print(cuda, sys.version)


True 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [ ]:
# Check if cuda is available
cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0
print("Cuda = "+str(cuda)+" with num_workers = "+str(num_workers))

Cuda = True with num_workers = 8


In [ ]:
from google.colab import drive
import json
drive.mount("/content/gdrive", force_remount=True)
%cd gdrive/MyDrive

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .

fatal: destination path 'ctcdecode' already exists and is not an empty directory.
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=9f9b42249c46ffc0335472c09d095a70f34c3ec6b9ee39481781351065df18b0
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
/content/gdrive/MyDrive/competitions/HW3P2/ctcdecode
Processing /content/gdrive/MyDrive/competitions/HW3P2/ctcdecode
  Created wheel for ctcdecode: filename=ctcdecode-1.0.2-cp37-cp37m-linux_x86_64.whl size=12848062 sha256=df12c2466f3fe46cd481d1b75bc97a05b73468cad711d8b3ee63bb55817b52df
  Stored in directory: /tmp/pip-ephem-wheel-cache-yh9xts8h/wheels/c9/45/15/144b3188a14cde7d9c8da247b43e7933387a4f89edbfdccad7
Successfully built ctcdecode


In [ ]:
# load phoneme
%cd  /content/gdrive/My Drive/competitions/HW3P2
from phoneme_list import PHONEME_MAP as phonemes
from phoneme_list import PHONEME_LIST as phonemes_list

/content/gdrive/My Drive/competitions/HW3P2


In [ ]:
!pip install python-Levenshtein
import Levenshtein

     |████████████████████████████████| 51kB 4.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149804 sha256=5a138a340d49f5f4a7cb5b7827451a6460a5a528ae88729f4227c70b3e838311
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


# define dataset & loader

In [ ]:
class MyDataset(data.Dataset):
  def __init__(self, X, Y):
    self.X = [torch.FloatTensor(x) for x in X]
    self.Y = [torch.LongTensor(y) for y in Y] 
    self.length = len(self.X)

  def __len__(self):
    return self.length

  def __getitem__(self, index):
    x = self.X[index]
    y = self.Y[index]
    
    return x,y

def collate(seq_list):
    X = [i[0] for i in seq_list]
    Y = [i[1] for i in seq_list]


    X_lens = torch.LongTensor([len(seq[0]) for seq in seq_list])
    Y_lens = torch.LongTensor([len(seq[1]) for seq in seq_list])

    X = pad_sequence(X)
    # `batch_first=True` is required for use in `nn.CTCLoss`.

    Y = pad_sequence(Y,batch_first=True) 

    return X,Y,X_lens,Y_lens

In [ ]:
class MyDataset_test(data.Dataset):
  def __init__(self, X):
    self.X = [torch.FloatTensor(x) for x in X]
    self.length = len(self.X)

  def __len__(self):
    return self.length

  def __getitem__(self, index):
    x = self.X[index]
    
    return x

def collate_test(seq_list):
    X = [i for i in seq_list]

    X_lens = torch.LongTensor([len(seq) for seq in seq_list])

    X = pad_sequence(X)

    return X,X_lens

In [ ]:
raw_train = np.load('./train.npy', allow_pickle=True)
raw_train_labels = np.load('./train_labels.npy', allow_pickle=True)
print("Raw training data shape:", raw_train.shape)
print("Raw training data label shape:", raw_train_labels.shape)

raw_dev = np.load('./dev.npy', allow_pickle=True)
raw_dev_labels = np.load('./dev_labels.npy', allow_pickle=True)
print("Raw dev data shape", raw_dev.shape)
print("Raw dev data label shape", raw_dev_labels.shape)

Raw training data shape: (14542,)
Raw training data label shape: (14542,)
Raw dev data shape (2200,)
Raw dev data label shape (2200,)


In [ ]:
train_dataset = MyDataset(raw_train, raw_train_labels)
del raw_train, raw_train_labels

dev_dataset = MyDataset(raw_dev, raw_dev_labels)
del raw_dev, raw_dev_labels

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64, collate_fn = collate,num_workers=0, pin_memory=True)
dev_loader = DataLoader(dev_dataset, shuffle=False, batch_size=64, collate_fn = collate,num_workers=0, pin_memory=True)

In [ ]:
raw_test = np.load('./test.npy', allow_pickle=True)
print("Raw test data shape:", raw_test.shape)

test_dataset = MyDataset_test(raw_test)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64, collate_fn = collate_test,num_workers=0, pin_memory=True)

Raw test data shape: (2561,)


# model setting

In [ ]:

class Model(nn.Module):
    def __init__(self):
      super(Model, self).__init__()
      self.cnn1 = torch.nn.Conv1d(40, 128, kernel_size=3, stride=1, padding=1,bias=False)
      self.bn1 = torch.nn.BatchNorm1d(128)
      self.cnn2 = torch.nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1,bias=False)
      self.bn2 = torch.nn.BatchNorm1d(128)

      self.cnn3 = torch.nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1,bias=False)
      self.bn3 = torch.nn.BatchNorm1d(256)


      self.relu = nn.ReLU(inplace=True)

      self.lstm1 = nn.LSTM(256, 256, bidirectional=True,num_layers=4,dropout=0.3)
      self.lr = nn.Linear(256*2, 1024)
      self.output = nn.Linear(1024, 42)



    def forward(self, X, lengths):

      X = self.cnn1(X.transpose(1,2))
      X = self.bn1(X)

      X = self.relu(X)

      X = self.cnn2(X)
      X = self.bn2(X)

      X = self.relu(X)
      
      X = self.cnn3(X)
      X = self.bn3(X)


      X = X.transpose(1,2)
      packed_X = pack_padded_sequence(input=X, lengths=lengths.cpu(), enforce_sorted=False)
      packed_out = self.lstm1(packed_X)[0]
      out, out_lens = pad_packed_sequence(packed_out)
      # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.

      out = self.relu(self.lr(out))
      out = self.output(out).log_softmax(2)
      return out, out_lens

In [ ]:
def reset_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                torch.nn.init.xavier_uniform_(m.weight.data)
            elif isinstance(m, nn.Linear):
                torch.nn.init.xavier_uniform_(m.weight.data)

network = Model()
network.apply(reset_params)
network = network.to(device)
print(network)

Model(
  (cnn1): Conv1d(40, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cnn3): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (lstm1): LSTM(256, 256, num_layers=4, dropout=0.3, bidirectional=True)
  (lr): Linear(in_features=512, out_features=1024, bias=True)
  (output): Linear(in_features=1024, out_features=42, bias=True)
)


# Training

In [ ]:
criterion = nn.CTCLoss() 
optimizer = torch.optim.Adam(network.parameters(), lr=2e-3,weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
numEpochs = 100

from ctcdecode import CTCBeamDecoder
decoder = CTCBeamDecoder(['$'] * 42, beam_width=21, log_probs_input=True)


In [ ]:
best_Dis =100000000

# network = torch.load('./model_6.pt')

# Validate
network.eval()
distance = 0
tot_sec = 0
avg_loss_val = 0
with torch.no_grad():
    for batch_idx, (X,Y,X_lens,Y_lens) in tqdm(enumerate(dev_loader),position=0, leave=True):
        X,Y,X_lens,Y_lens = X.to(device),Y.to(device),X_lens.to(device),Y_lens.to(device)
        out, out_lens = network(X, X_lens)

        loss = criterion(out, Y, out_lens, Y_lens)
        avg_loss_val += loss.item()
        out = out.cpu()
        out_lens = out_lens.cpu()
        test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0, 1), out_lens)
        for i in range(out.size()[1]):
            best_seq = test_Y[i, 0, :test_Y_lens[i, 0]]
            best_pron = ''.join(phonemes[i] for i in best_seq)
            best_tar = ''.join(phonemes[j] for j in Y[i] if j!=0)
            distance += Levenshtein.distance(best_pron, best_tar)
            tot_sec += 1
    avg_loss_validation = avg_loss_val / len(dev_loader)
    best_Dis = distance/tot_sec
    print('best_Dis',best_Dis,'avg_loss_validation',avg_loss_validation)
    del X,Y,X_lens,Y_lens,out, out_lens,loss




for epoch in range(numEpochs):
    # train
    network.train()
    avg_loss = 0.0 
    for batch_num, (X,Y,X_lens,Y_lens) in tqdm(enumerate(train_loader),position=0, leave=True):
      optimizer.zero_grad()
      X,Y,X_lens,Y_lens = X.to(device),Y.to(device),X_lens.to(device),Y_lens.to(device)

      out, out_lens = network(X, X_lens)
      loss = criterion(out, Y, out_lens, Y_lens)
      avg_loss += loss.item()
      loss.backward()
      optimizer.step()
      if batch_num % 50 == 49:
          print('Epoch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, avg_loss/50))
          avg_loss = 0.0
      del X,Y,X_lens,Y_lens,out, out_lens,loss


    # Validate
    network.eval()
    distance = 0
    tot_sec = 0
    avg_loss_val = 0
    with torch.no_grad():
        for batch_idx, (X,Y,X_lens,Y_lens) in tqdm(enumerate(dev_loader),position=0, leave=True):
            X,Y,X_lens,Y_lens = X.to(device),Y.to(device),X_lens.to(device),Y_lens.to(device)
            out, out_lens = network(X, X_lens)

            loss = criterion(out, Y, out_lens, Y_lens)
            avg_loss_val += loss.item()
            out = out.cpu()
            out_lens = out_lens.cpu()
            test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0, 1), out_lens)
            for i in range(out.size()[1]):
                best_seq = test_Y[i, 0, :test_Y_lens[i, 0]]
                best_pron = ''.join(phonemes[i] for i in best_seq)
                best_tar = ''.join(phonemes[j] for j in Y[i] if j!=0)
                distance += Levenshtein.distance(best_pron, best_tar)
                tot_sec += 1
        avg_loss_validation = avg_loss_val / len(dev_loader)
        avg_dist = distance/tot_sec
        print('Epoch: {}, Validation loss: {:.2f}'.format(epoch, avg_loss_validation))
        print('Epoch: {}, Validation Distance: {:.2f}'.format(epoch, avg_dist))

        scheduler.step(avg_loss_validation)
        print('Epoch {}, lr {}'.format(epoch, optimizer.param_groups[0]['lr']))
        del X,Y,X_lens,Y_lens,out, out_lens,loss

    if avg_dist < best_Dis:
      best_Dis = avg_dist
      print('val Dist',avg_dist)
      torch.save(network, 'model_6.pt')




35it [01:30,  2.58s/it]
0it [00:00, ?it/s]

best_Dis 263.56363636363636 avg_loss_validation 32.07630375453404


50it [01:07,  1.35s/it]

Epoch: 1	Avg-Loss: 4.8584


100it [02:13,  1.33s/it]

...

228it [05:03,  1.33s/it]
35it [01:29,  2.56s/it]
0it [00:00, ?it/s]

Epoch: 28, Validation loss: 0.48
Epoch: 28, Validation Distance: 7.86
Epoch 28, lr 0.0005
val Dist 7.863181818181818


50it [01:07,  1.35s/it]

Epoch: 30	Avg-Loss: 0.0774


51it [01:08,  1.34s/it]

# Test output

In [ ]:
network = torch.load('./model_6.pt')

decoder = CTCBeamDecoder(['$'] * 42, beam_width=100, log_probs_input=True)


output = []
network.eval()
with torch.no_grad():
    for batch_idx, (X,X_lens) in tqdm(enumerate(test_loader),position=0, leave=True):
        X,X_lens = X.to(device),X_lens.to(device)
        out, out_lens = network(X, X_lens)
        out = out.cpu()
        out_lens = out_lens.cpu()
        test_Y, _, _, test_Y_lens = decoder.decode(out.transpose(0, 1), out_lens)
        for i in range(out.size()[1]):
            best_seq = test_Y[i, 0, :test_Y_lens[i, 0]]
            best_pron = ''.join(phonemes[i] for i in best_seq)
            output.append(best_pron)
    del X,X_lens,out,out_lens

print(output[0:5])

41it [08:19, 12.18s/it]

['.prelhkoRdiNlIRAnthDhbOindOAtDhfrDrendhvDhHol.AndlukthloNDhvisthvhgaRdhnWok.kaRphtid.WiTklOslISEvhngRAs.hndboRdrdWiTshmRUdAndinmrtoR.htempt.htRhbeRI.', '.DhpwldhpvEprzkhntenstinhWotr.AndDIeR.putinhvylhntAkShntUshplyDhvAkim.leftbyDhkandhnsEShn.hvDhmist.RwzizitselfintUhWrldWen.', '.AndfiRsDwbikhzyvAniS.AndAmsInOmoR.', '.fAdhm.siksfIt.', '.DhveRIbilhtIhvmhlthfhlpaRts.']


In [ ]:
with open("trial.csv", 'w') as fh:
  fh.write('id,label\n') 
  for i in range(len(output)):
    fh.write(str(i)+ ',' + output[i] + "\n")

In [ ]:
import json
token = {"username":"minxuej","key":"fd1bdd010edd811c494aa3c4d32b2bd7"}
with open('/content/gdrive/MyDrive/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp /content/gdrive/MyDrive/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content/gdrive/MyDrive}
!chmod 600 /root/.kaggle/kaggle.json

- path is now set to: {/content/gdrive/MyDrive}


In [ ]:
# Use Kaggle API to submit your result
!kaggle competitions submit -c 11785-spring2021-hw3p2 -f trial.csv -m "earl"

100% 204k/204k [00:05<00:00, 37.7kB/s]
Successfully submitted to 11785 Homework 3 Part 2: Seq to Seq